In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from keras.models import *
from keras.layers import *
from keras import initializers
import sys
import numpy as np
import h5py
import tensorflow as tf
sys.path.append('/home/mk2299/Mice_experiments/attorch')
sys.path.append('/home/mk2299/Mice_experiments/Sinz2018_NIPS/')
sys.path.append('/home/mk2299/Mice_experiments/Sinz2018_NIPS/my_exp/')
from scipy import stats
import tensorflow.contrib.slim as slim
import hashlib
import inspect
import random
from tensorflow import losses
from numpy import pi
from collections import OrderedDict
from keras import optimizers
#from attorch.losses import PoissonLoss3d
from base_models import Readout, smoothness_regularizer_2d, group_sparsity_regularizer_2d
from nips2018.architectures.readouts import SpatialTransformerPooled3dReadout, ST3dSharedGridStopGradientReadout, FullyConnectedReadout
from nips2018.architectures.cores import StackedFeatureGRUCore, Stacked3dCore, Stacked2dCore
from nips2018.architectures.base import CorePlusReadout3d
from attorch.layers import elu1, Elu1
import torch.nn as nn
from torch.nn import functional as F
import torch
import pickle
from itertools import count
from tqdm import tqdm
from collections import namedtuple
from attorch.train import early_stopping, cycle_datasets
from itertools import chain, repeat
from nips2018.utils.measures import corr
from scipy.stats import pearsonr
from models_attention import TimeDistributed, ConvLSTM
from models.resnet import generate_model

Using TensorFlow backend.
/home/mk2299/Mice_experiments/Sinz2018_NIPS/nips2018/architectures/cores.py:18: UserWarning: Could not import CajalUnit. You won't be able to use the NetGardCore
  warn("Could not import CajalUnit. You won't be able to use the NetGardCore")


In [2]:
def ResNet_core(freeze = False, pretrained = False):
    core = generate_model(model_depth = 18) 
    if pretrained:
        pretrain_mdl = torch.load('./pretrained_weights/r3d18_K_200ep.pth')
        core.load_state_dict(pretrain_mdl['state_dict'])
    if freeze:
        for param in core.parameters():
            param.requires_grad = False
    return core

In [3]:
core = ResNet_core()

In [4]:
n_neurons = OrderedDict([('0',1740)])#[(k, v.n_neurons) for k, v in f['neurons'].items()])

readout = ST3dSharedGridStopGradientReadout(torch.Size([64, 150,18,32]),#[64, 75, 9, 16] #torch.Size([64, 150, 18,32]), 
                                               n_neurons, 
                                               positive=False,  
                                               gamma_features=1., 
                                               pool_steps=2,
                                                kernel_size=4,
                                                stride=4,
                                            gradient_pass_mod=3
                                           )


---------------------------+--------------------------------------------------------------------------------
ST3dSharedGridStopGrad...  | Ignoring input {} when creating ST3dSharedGridStopGradientReadout
ST3dSharedGridStopGrad...  | 	Neuron change detected from -1 to 1740 ! Resetting grid!
ST3dSharedGridStopGrad...  | 	Gradient for 0 will pass


In [5]:
model = CorePlusReadout3d(core, readout, nonlinearity=Elu1(), 
                        shifter=None, modulator=None, burn_in=15)


In [6]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

33902304

# Load data

In [7]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, sets = 'train'):
        'Initialization'
        self.sets = sets
        with open('/home/mk2299/Mice_experiments/Sinz2018_NIPS/Sinz2018_NIPS_data/cache/17358-5-3.pkl', 'rb') as pkl_file:
            self.data_dict = pickle.load(pkl_file)
        self.list_IDs = np.arange(len(self.data_dict['stims_' + sets]))

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = self.data_dict['stims_' + self.sets][ID]
        y = self.data_dict['responses_' + self.sets][ID]
        start_idx = np.random.randint(X.shape[1]-151)
        return np.repeat(X[:,start_idx:start_idx + 150], 3, 0), (y[start_idx:start_idx + 150])#[::2] #X[:,start_idx:start_idx + 150]

# Load training and validation generators

In [8]:
params = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 6}

# Generators
training_set = Dataset(sets = 'train')
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = Dataset(sets = 'val')
validation_generator = torch.utils.data.DataLoader(validation_set, **params)


_Loss function_

In [9]:
class PoissonLoss3d(nn.Module):
    def __init__(self, bias=1e-16, per_neuron=False):
        super().__init__()
        self.bias = bias
        self.per_neuron = per_neuron

    def forward(self, output, target):
        #_assert_no_grad(target)
        lag = target.size(1) - output.size(1)
        loss =  (output - target[:, lag:, :] * torch.log(output + self.bias))
        if not self.per_neuron:
            return loss.mean()
        else:
            return loss.view(-1, loss.shape[-1]).mean(dim=0)

In [10]:
img_shape = training_set.__getitem__(0)[0] #list(trainloaders.values())[0].dataset.img_shape



criterion = PoissonLoss3d()
n_datasets = training_set.__len__() #len(trainloaders)
acc = 1 # accumulate gradient over this many steps


# --- setup objective
grad_passes = 0
for ro in model.readout.values():
    grad_passes += int(not ro.stop_grad)


mu_dict = OrderedDict([
    (k, torch.Tensor(np.load('/home/mk2299/Mice_experiments/Sinz2018_NIPS/Sinz2018_NIPS_data/cache/17358-5-3_mean.npy'))) for k in ['0']
])
model.readout.initialize(mu_dict)
#model.core.initialize()

model = model.cuda()

ST3dSharedGridStopGrad...  | Initializing with mu_dict: 0: 1740


In [11]:
def full_objective(model, readout_key, inputs, targets):
    outputs = model(inputs, '0')
    return (criterion(outputs, targets)
            #+ (model.core.regularizer() / grad_passes if not model.readout[readout_key].stop_grad else 0)
            + model.readout.regularizer(readout_key).cuda(0)) / acc


In [12]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
PerformanceScores = namedtuple('PerformanceScores', ['pearson'])


def compute_scores(y, y_hat, axis=0):
    pearson = corr(y, y_hat, axis=axis)
    return PerformanceScores(pearson=pearson)

def compute_predictions(loader, model, readout_key = '0', reshape=True, stack=True, return_lag=False):
    y, y_hat = [], []
    for x_val, y_val in loader:
        neurons = y_val.size(-1)

        y_mod = model(x_val.cuda(), readout_key).data.cpu().numpy()
        

        lag = y_val.shape[1] - y_mod.shape[1]
        if reshape:
            y.append(y_val[:, lag:, :].numpy().reshape((-1, neurons)))
            y_hat.append(y_mod.reshape((-1, neurons)))
        else:
            y.append(y_val[:, lag:, :].numpy())
            y_hat.append(y_mod)
    if stack:
        y, y_hat = np.vstack(y), np.vstack(y_hat)
    if not return_lag:
        return y, y_hat
    else:
        return y, y_hat, lag

In [14]:
model.train()
accumulate_gradient=1
schedule = [0.005]#, 0.001]
n_epochs = 100

for opt, lr in zip(repeat(torch.optim.Adam), schedule):
    print('Training with learning rate', lr)
    optimizer = opt(model.parameters(), lr=lr)
    optimizer.zero_grad()
    iteration = 0
    assert accumulate_gradient > 0, 'accumulate_gradient needs to be > 0'
    for epoch in range(n_epochs):
        for x_batch, y_batch in training_generator:
            obj = full_objective(model, '0', x_batch.cuda(), y_batch.cuda())
            obj.backward()
            if iteration % accumulate_gradient == accumulate_gradient - 1:
                optimizer.step()
                optimizer.zero_grad()
            iteration += 1 
            if iteration % 100 == 0:
                model.eval()
                true, preds = compute_predictions(validation_generator, model)
                print('Val correlation:', np.mean([pearsonr(true[:,i], preds[:,i])[0] for i in range(true.shape[1])]))
                model.train()

Training with learning rate 0.005
Val correlation: 0.04952668
Val correlation: 0.04991383
Val correlation: 0.061513335
Val correlation: 0.06991729
Val correlation: 0.076648004
Val correlation: 0.08544331
Val correlation: 0.07611467
Val correlation: 0.08753918
Val correlation: 0.090169095
Val correlation: 0.09248983
Val correlation: 0.098343894
Val correlation: 0.09784792
Val correlation: 0.1036094
Val correlation: 0.09947476
Val correlation: 0.10766456
Val correlation: 0.10927443
Val correlation: 0.11425991
Val correlation: 0.109888606
Val correlation: 0.118538536
Val correlation: 0.12194548
Val correlation: 0.12053082
Val correlation: 0.12201126
Val correlation: 0.12138126
Val correlation: 0.12213685
Val correlation: 0.12482516
Val correlation: 0.12357618
Val correlation: 0.11969582
Val correlation: 0.12225707
Val correlation: 0.12547177
Val correlation: 0.11803852
Val correlation: 0.12391202
Val correlation: 0.122624576
Val correlation: 0.12941213
Val correlation: 0.13414761
Val corr

KeyboardInterrupt: 

# Compute performance on validation

In [15]:
model.eval()
true, preds = compute_predictions(validation_generator, model)
print(np.mean([pearsonr(true[:,i], preds[:,i])[0] for i in range(true.shape[1])]))

0.14506392


# Test set performance

In [16]:
torch.save(model, "final_models/17358-5-3_3DResNet_first_block.pt")
model.eval()

CorePlusReadout3d(
  (core): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, mom

In [17]:
class Dataset_test(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self):
        'Initialization'
        
        with open('/home/mk2299/Mice_experiments/Sinz2018_NIPS/Sinz2018_NIPS_data/cache/17358-5-3_test.pkl', 'rb') as pkl_file:
            self.data_dict = pickle.load(pkl_file)
        self.list_IDs = np.arange(len(self.data_dict['stims_test']))

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = self.data_dict['stims_test'][ID]
        y = self.data_dict['responses_test'][ID]
        
        return np.repeat(X,3,0), y  

In [18]:
PerformanceScores = namedtuple('PerformanceScores', ['pearson'])


def compute_scores(y, y_hat, axis=0):
    pearson = corr(y, y_hat, axis=axis)
    return PerformanceScores(pearson=pearson)

def compute_predictions(loader, model, readout_key = '0', reshape=True, stack=True, return_lag=False):
    y, y_hat = [], []
    for x_val, y_val in loader:
        neurons = y_val.size(-1)
        for t in range(x_val.size(2)//150):
            y_mod = model(x_val[:,:,t*150:(t+1)*150].cuda(), readout_key).data.cpu().numpy()
            lag = y_val[:,t*150:(t+1)*150].shape[1] - y_mod.shape[1] 
            if reshape:
                y.append((y_val[:,t*150:(t+1)*150])[:, lag:, :].numpy().reshape((-1, neurons)))
                y_hat.append(y_mod.reshape((-1, neurons)))
            else:
                y.append((y_val[:,t*150:(t+1)*150])[:, lag:, :].numpy())
                y_hat.append(y_mod)
    if stack:
        y, y_hat = np.vstack(y), np.vstack(y_hat)
    if not return_lag:
        return y, y_hat
    else:
        return y, y_hat, lag

In [19]:
params = {'batch_size': 1,
          'shuffle': False,
          'num_workers': 6}
test_set = Dataset_test()
test_generator = torch.utils.data.DataLoader(test_set, **params)

In [20]:
true, preds = compute_predictions(test_generator, model)
corrs = np.array([pearsonr(true[:,i], preds[:,i])[0] for i in range(true.shape[1])])
print(np.nanmean(corrs))

0.124457456


# Plots

In [ ]:
corrs = {}
y = []
x = ['GRU',  'static','LSTM']
for name in x:
    corrs[name] = np.mean(np.load('../17358-5-3-' + name + '-corrs.npy'))
    y.append(np.mean(np.load('../17358-5-3-' + name + '-corrs.npy')))

In [ ]:
corrs['GRU']

In [ ]:
y = [0.128, 0.095]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(4,5))
sns.set(font_scale=1.2, font = 'arial')
sns.set_style('white')
plt.title('Prediction accuracy (R) : 17797-8-5')
sns.barplot(x, y); 
plt.tight_layout()
plt.savefig('17797-8-5_static.png', dpi = 200)

# Recurrent

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.2, font = 'arial')
sns.set_style('white')

fig, ax = plt.subplots(figsize = (6,6))
N = 2
ind = np.arange(N)    # the x locations for the groups
width = 0.35         # the width of the bars
p1 = ax.bar(ind, [0.085, 0.101], width, label = 'Pretrained frozen')
p2 = ax.bar(ind + width, [0.102, 0.117], width, label = 'Direct fit')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('conv1', 'block1'))
plt.ylabel('Pearson correlation')
plt.legend()
plt.title('3D ConvNets')
plt.tight_layout()
plt.savefig('17358.png', dpi = 200)